In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
#LSTM Cell
class LSTMCell(tf.keras.layers.Layer):
  def __init__(self):
    super(LSTMCell, self).__init__()

    #gates as dense layers with respective activation function
    self.forget_gate = tf.keras.layers.Dense(units = 2 , activation = tf.keras.activations.sigmoid, bias_initializer='ones')
    self.input_gate = tf.keras.layers.Dense(units = 2, activation = tf.keras.activations.sigmoid)
    self.output_gate = tf.keras.layers.Dense(units = 2, activation = tf.keras.activations.sigmoid)
    
    #cell state candidates with tanh
    self.candidates = tf.keras.layers.Dense(units = 2, activation = tf.keras.activations.tanh)

    self.tanh = tf.keras.layers.tanh()

  def call(self, input, (hidden_state, cell_state)):
    #concatenate input and hidden state
    concat_input = tf.keras.layers.concatenate(hidden_state,input)

    #forget part of cell state
    cell_state = cell_state * self.forget_gate(concat_input)

    #compute update for cell state
    update = self.input_gate(concat_input) * self.candidates(concat_input)
    #use input gate and candidates to update cell state
    cell_state = cell_state + update

    #compute new hidden state
    hidden_state = ouput_gate(concat_input) * self.tanh(cell_State)

    #ouput the hidden state
    return hidden_state

In [ ]:
class LSTM(tf.keras.layers.Layer):
  def __init__(self):
    self.cell = LSTMCell()


  def call(self, input):
    self.cell(input)
    #input shpae: [batchsize, timesteps, inputsize
    return 

In [ ]:
class Model():
  

In [ ]:
#generate sequence
def generate_sequence(max_length):
  np.array
  return